##### Importing Necessary Packages

In [29]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

##### Loading the Data

In [11]:
data = pd.read_csv('../data/MachineLearningRating_v3.txt', delimiter='|')

C:\Users\Gedion Afework\AppData\Local\Temp\ipykernel_13300\1502387084.py:1: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/MachineLearningRating_v3.txt', delimiter='|')


In [12]:
data.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [13]:
data.columns

Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode',
       'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors',
       'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser',
       'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff',
       'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet',
       'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm',
       'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section',
       'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium',
       'TotalClaims'],
      dtype='object')

##### Handle Missing Values

In [15]:
numerical_cols = ['TotalPremium', 'TotalClaims', 'CustomValueEstimate', 'Cylinders', 'cubiccapacity', 'kilowatts', 'SumInsured']
for col in numerical_cols:
    if data[col].isnull().sum():
        data[col] = data[col].fillna(data[col].median())
    else:
        continue

In [17]:
categorical_cols = ['Province', 'Gender', 'VehicleType', 'make', 'CoverType']
for col in categorical_cols:
    data[col] = data[col].fillna('Unknown')

##### Feature Engineering

In [18]:
data['VehicleAge'] = 2025 - data['RegistrationYear']
data['HasClaim'] = data['TotalClaims'] > 0
data['PremiumToValue'] = data['TotalPremium'] / data['CustomValueEstimate'].replace(0, 1)
data['Margin'] = data['TotalPremium'] - data['TotalClaims']

In [19]:
data['VehicleAge'].head()

0    21
1    21
2    21
3    21
4    21
Name: VehicleAge, dtype: int64

In [20]:
data['HasClaim'].head()

0    False
1    False
2    False
3    False
4    False
Name: HasClaim, dtype: bool

In [21]:
data['PremiumToValue'].head()

0    0.000184
1    0.000184
2    0.000000
3    0.004299
4    0.000000
Name: PremiumToValue, dtype: float64

In [22]:
data['Margin'].head()

0     21.929825
1     21.929825
2      0.000000
3    512.848070
4      0.000000
Name: Margin, dtype: float64

In [23]:
data.to_csv('../data/processed_data.csv', index=False)

In [27]:
encoder = OneHotEncoder(drop='first', handle_unknown='ignore')
encoded_cols = encoder.fit_transform(data[categorical_cols])
encoded_data = pd.DataFrame.sparse.from_spmatrix(encoded_cols, columns=encoder.get_feature_names_out())
data_encoded = pd.concat([data.drop(columns=categorical_cols), encoded_data], axis=1)

In [28]:
data_encoded.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,CoverType_Keys and Alarms,CoverType_Own Damage,CoverType_Passenger Liability,CoverType_Roadside Assistance,CoverType_Signage and Vehicle Wraps,CoverType_Standalone passenger liability,CoverType_Third Party,CoverType_Third Party Only,CoverType_Trailer,CoverType_Windscreen
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,0,0,0,0,0,0,0,0,0,1.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,0,0,0,0,0,0,0,0,0,1.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,0,0,0,0,0,0,0,0,0,1.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,0,1.0,0,0,0,0,0,0,0,0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,0,1.0,0,0,0,0,0,0,0,0


##### Train-Test Split

In [30]:

X = data_encoded.drop(['TotalClaims', 'CalculatedPremiumPerTerm', 'HasClaim', 'Margin'], axis=1)
y_severity = data_encoded[data_encoded['HasClaim'] == True]['TotalClaims']
y_premium = data_encoded['CalculatedPremiumPerTerm']
y_claim = data_encoded['HasClaim']
X_severity = X[data_encoded['HasClaim'] == True]
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_severity, y_severity, test_size=0.2, random_state=42)
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X, y_premium, test_size=0.2, random_state=42)
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_claim, test_size=0.2, random_state=42)